In [1]:
import time
import gc
import os
# import sys
from optimum.onnxruntime import ORTModelForCausalLM
from transformers import set_seed, AutoTokenizer, AutoModelForCausalLM, pipeline
from pathlib import Path
import torch
import gc
import gguf
from smoothquant.smooth import smooth_lm
from smoothquant.calibration import get_act_scales

c:\ProgramData\anaconda3\envs\base1\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\ProgramData\anaconda3\envs\base1\lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\ProgramData\anaconda3\envs\base1\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Defining Paths

In [2]:
# Paths
HFPATH = "HuggingFaceH4/zephyr-7b-beta"
MODELS = Path.cwd() / "models"
zephyr_HF = MODELS / "zephyr_HF"
zephyr_HF_onnx = MODELS / "onnx"
QUANT_DATA_PATH = Path.cwd() / "datasets" / "quant_data" / "val.jsonl"
VAIP_CONFIG = Path.cwd() / "vaip_config_transformers.json"
# zephyr_gguf = Path.cwd() / "models" / "zephyr-7b-beta.Q4_K_M.gguf"

for folder in [MODELS, zephyr_HF, zephyr_HF_onnx]:
    folder.mkdir(exist_ok=True, parents=True)

# Loading Models

In [3]:
# # Zephyr GGUF
# model_id = zephyr_gguf.parent
# filename = zephyr_gguf.name
# tokenizer = AutoTokenizer.from_pretrained(model_id, gguf_file=filename)
# model = AutoModelForCausalLM.from_pretrained(model_id, gguf_file=filename)

In [4]:
# Zephyr Model
# model = AutoModelForCausalLM.from_pretrained(zephyr_HF)
# model.save_pretrained(zephyr_HF) # Saving the model to folder


In [5]:
# Tokenizers
# tokenizer = AutoTokenizer.from_pretrained(HFPATH)
# tokenizer = AutoTokenizer.from_pretrained(zephyr_HF / "tokenizer")
# tokenizer.save_pretrained(zephyr_HF / "tokenizer") # Saving the tokenizer to folder

In [6]:
# Inference with zephyr
# zephyr_chat = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.bfloat16, device_map="auto")
text_input = "Give me a small funny story in 3 sentences."
messages = [
    {
        "role": "system",
        "content": "You are a friendly AI companion who helps the user with tasks and also converses with them as a chatbot when required.",
    },
    {
        "role": "user",
        "content": text_input,
    },
]
# prompt = zephyr_chat.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# text_output = zephyr_chat(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
# text_output[0]["generated_text"]
# del zephyr_chat
# gc.collect()

# Performing Smooth Quantization

In [7]:
# act_scales = get_act_scales(model, tokenizer, str(QUANT_DATA_PATH))
# ortmodel = ORTModelForCausalLM.from_pretrained(HFPATH)

# Loading ONNX model with VitisAI

In [8]:
provider="VitisAIExecutionProvider"
provider_options = {"config_file": VAIP_CONFIG}
tokenizer = AutoTokenizer.from_pretrained(HFPATH)
model = ORTModelForCausalLM.from_pretrained(zephyr_HF_onnx, provider=provider, provider_options=provider_options)

: 

In [ ]:
# Inference with zephyr on ONNX
zephyr_chat = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.bfloat16, device_map="auto")
text_input = "Give me a small funny story in 3 sentences."
messages = [
    {
        "role": "system",
        "content": "You are a friendly AI companion who helps the user with tasks and also converses with them as a chatbot when required.",
    },
    {
        "role": "user",
        "content": text_input,
    },
]
prompt = zephyr_chat.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
text_output = zephyr_chat(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
text_output[0]["generated_text"]
# del zephyr_chat
# gc.collect()